In [ ]:
##Same as the Standard GPH file for clustering but instead 
##... cluster on anomalies of wintertime gph.

In [ ]:
#import packages
import datetime as dt
import numpy as np
import matplotlib 
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import cartopy.feature
from cartopy.util import add_cyclic_point
import cartopy.crs as ccrs
import xarray as xr
import scipy.stats
import pickle

#import statements for the clustering-related portions
from sklearn.cluster import KMeans as km

In [ ]:
#open files for split gph data and recombine
infile = open("../../New_ERAfiles/gph_NEW.p", 'rb')
gph = pickle.load(infile)
infile.close()

infile = open("../../New_ERAfiles/gph_time_NEW.p", 'rb')
gph_time = pickle.load(infile)
infile.close()

infile = open("../../New_ERAfiles/gph_lat_NEW.p", 'rb')
gph_lat = pickle.load(infile)
infile.close()

infile = open("../../New_ERAfiles/gph_lon_NEW.p", 'rb')
gph_lon = pickle.load(infile)
infile.close()

In [ ]:
gph.shape

In [ ]:
#non-stacked
gph = xr.DataArray(data= gph, 
                       dims = ["time","lat","lon"],
                       coords = dict(
                           time = gph_time,
                           lat = gph_lat,
                           lon = gph_lon)
                      )

In [ ]:
#calculate daily mean values
daily_mean_gph = gph.groupby('time.dayofyear').mean()

In [ ]:
#test gph anomaly
daily_anom_gph = gph.groupby('time.dayofyear') - daily_mean_gph

In [ ]:
daily_anom_gph

In [ ]:
anom_D = daily_anom_gph.groupby('time.date').mean()

In [ ]:
##full_S = full_D.stack(mtime = ("metric","date"), latlon = ("lat","lon")) #stack lat and lon
anom_S2 = daily_anom_gph.stack(latlon = ("lat","lon")) #stack lat and lon
anom_S2

In [ ]:
##full_S = full_D.stack(mtime = ("metric","date"), latlon = ("lat","lon")) #stack lat and lon
anom_S = anom_D.stack(latlon = ("lat","lon")) #stack lat and lon
anom_S

In [ ]:
inertia = []
clusters = np.arange(1,9,1) #testing number of clusters 
for n in clusters:
    #provides labels of clusters
    h_km = km(n_clusters = n).fit(anom_S2)
    #Getting unique labels
    h_labels = np.unique(h_km)
    h_inertia = h_km.inertia_
    inertia.append(h_inertia)

In [ ]:
%matplotlib inline
plt.title("Cluster Inertia Test")
plt.scatter(clusters,inertia)
plt.xlabel("N Clusters")
plt.ylabel("Cluster Inertia")
plt.show()

In [ ]:
from sklearn.decomposition import PCA
 
pca = PCA(5)

In [ ]:
test = pca.fit_transform(anom_S2)

In [ ]:
#plotting the results:
h_km = km(n_clusters = 5, random_state=0)
label = h_km.fit_predict(anom_S2)
#Getting unique labels
h_labels = np.unique(label)

In [ ]:
%matplotlib inline
for i in h_labels:
    plt.scatter(test[label == i , 0] , test[label == i , 1] , label = i)
plt.legend()
plt.show()

In [ ]:
label

In [ ]:
pickle.dump(label, open("ALL_HOURLY_ANOMcluster_labels.p", 'wb'))

In [ ]:
infile = open("UW_lat.p", 'rb')
unweighted_lat = pickle.load(infile)
infile.close()

infile = open("UW_lon.p", 'rb') 
unweighted_lon = pickle.load(infile)
infile.close()

In [ ]:
#initialize separation of four clusters
C1 = []
C2 = []
C3 = []
C4 = []
C5 = []

for k in range(len(label)):
    if label[k] == 0:
        C1.append("k")
    if label[k] == 1:
        #print(k)
        C2.append("k")
    if label[k] == 2:
        #print(k)
        C3.append("k")
    if label[k] == 3:
        #print(k)
        C4.append("k") 
    if label[k] == 4:
        #print(k)
        C5.append("k") 

print(len(C1))
print(len(C2))
print(len(C3))
print(len(C4)) 
print(len(C5)) 

In [ ]:
C1 = np.zeros((3510,128,512))
C2 = np.zeros((2660,128,512))
C3 = np.zeros((2462,128,512))
C4 = np.zeros((1197,128,512))
C5 = np.zeros((2331,128,512))

In [ ]:
#set the initial index for each empty array to 0 so it can increas with each aded value
index1 = 0
index2 = 0
index3 = 0
index4 = 0
index5 = 0

#loop, create clustered arrays, yay
for k in range(len(label)):
    if label[k] == 0:
        C1[index1,:,:] = gph[k,:,:]
        index1 = index1 + 1
        
    if label[k] == 1:
        C2[index2,:,:] = gph[k,:,:]
        index2 = index2 + 1
        
    if label[k] == 2:
        C3[index3,:,:] = gph[k,:,:]
        index3 = index3 + 1
        
    if label[k] == 3:
        C4[index4,:,:] = gph[k,:,:]
        index4 = index4 + 1
 
    if label[k] == 4:
        C5[index5,:,:] = gph[k,:,:]
        index5 = index5 + 1

In [ ]:
lat_index = unweighted_lat
lon_index = unweighted_lon

In [ ]:
h_cluster1 = xr.DataArray(data= np.mean(C1,axis=0), 
                       dims = ["lat","lon"],
                       coords = dict(
                           lat = lat_index,
                           lon = lon_index)
                      )

h_cluster2 = xr.DataArray(data= np.mean(C2,axis=0), 
                       dims = ["lat","lon"],
                       coords = dict(
                           lat = lat_index,
                           lon = lon_index)
                      )

h_cluster3 = xr.DataArray(data= np.mean(C3,axis=0), 
                       dims = ["lat","lon"],
                       coords = dict(
                           lat = lat_index,
                           lon = lon_index)
                      )

h_cluster4 = xr.DataArray(data= np.mean(C4,axis=0), 
                       dims = ["lat","lon"],
                       coords = dict(
                           lat = lat_index,
                           lon = lon_index)
                      )

h_cluster5 = xr.DataArray(data= np.mean(C5,axis=0), 
                       dims = ["lat","lon"],
                       coords = dict(
                           lat = lat_index,
                           lon = lon_index)
                      )

In [ ]:
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import matplotlib.ticker as mticker
import matplotlib.colors
import matplotlib.cm

plt.Figure(figsize=(20,20),dpi=120)
ax = plt.axes(projection=ccrs.Orthographic(0,90))
ax.add_feature(cartopy.feature.LAND, zorder=1, edgecolor='dimgray',facecolor='none')
#ax.set_extent([-180,180,0,90],ccrs.PlateCarree())
ax.outline_patch.set_edgecolor('none')
gl = ax.gridlines(draw_labels=True,linewidth=1, color='black', alpha=0.5)
gl.xlabels_top = True
gl.ylabels_left = False
gl.ylabels_right = False
gl.xlines = True
gl.xformatter = LONGITUDE_FORMATTER
gl.yformatter = LATITUDE_FORMATTER
gl.xlabel_style = {'size': 10, 'color': 'black'}

ax.contourf(lon_index,lat_index,h_cluster5,transform=ccrs.PlateCarree(),cmap="bwr")

cmap = matplotlib.cm.bwr
norm = matplotlib.colors.Normalize(vmin = np.min(anom_S2.values),vmax=np.max(anom_S2.values))

cbar = plt.colorbar(matplotlib.cm.ScalarMappable(norm=norm, cmap=cmap), ax=ax, location = 'right', shrink=0.6, pad=0.1) #add colorbar
#tick_locator = ticker.MaxNLocator(nbins=10)
#cbar.locator = tick_locator
#cbar.update_ticks()
cbar.set_label(label='Geopotential Height (m)', size=10)
cbar.ax.tick_params(labelsize=10)

plt.tight_layout
plt.title("ERA-I 10hPa Geopotential Height Anomalies, Cluster 5")
plt.savefig("ALL_HOURLY_ANOMgph_C5map.png")